# Split big geojson into regions

## Setup

In [1]:
import pandas as pd
import numpy as np

# For importing geojson:
import json

# For saving new geojson:
from geojson import FeatureCollection

# For drawing maps:
import folium
from folium import plugins

## Import data

In [2]:
df_nearest_hospitals = pd.read_csv('./lsoa_nearest_hospitals.csv')

df_nearest_hospitals.head()

,LSOA11CD,LSOA,Nearest hospital postcode,Nearest hospital name,Time (mins) to nearest hospital,Nearest MT hospital postcode,Nearest MT hospital name,Time (mins) to nearest MT hospital
0,E01031349,Adur 001A,BN25BE,"Royal Sussex County Hospital, Brighton",17.6,BN25BE,"Royal Sussex County Hospital, Brighton",17.6
1,E01031350,Adur 001B,BN112DH,Worthing Hospital,18.7,BN25BE,"Royal Sussex County Hospital, Brighton",18.7
2,E01031351,Adur 001C,BN112DH,Worthing Hospital,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",19.8
3,E01031352,Adur 001D,BN112DH,Worthing Hospital,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",19.8
4,E01031370,Adur 001E,BN112DH,Worthing Hospital,16.5,BN25BE,"Royal Sussex County Hospital, Brighton",19.8


Find all of the unique hospital postcodes:

In [3]:
ivt_hospital_postcodes_set = set(df_nearest_hospitals['Nearest hospital postcode'])
mt_hospital_postcodes_set = set(df_nearest_hospitals['Nearest MT hospital postcode'])

In [4]:
len(ivt_hospital_postcodes_set), len(mt_hospital_postcodes_set)

(125, 36)

For each of these STP and LHB codes, find the LSOAs that belong to it:

In [5]:
# Record these for info or naming of the .geojson files:
code_dict = {}
hosptial_name_dict = {}

# Record these just out of interest:
ivt_hospital_names_list = []
n_lsoas_list = []

for postcode in ivt_hospital_postcodes_set:
    df_nearest_hospitals_here = df_nearest_hospitals[df_nearest_hospitals['Nearest hospital postcode'] == postcode]
    LSOA_codes_here = df_nearest_hospitals_here['LSOA'].values
    
    code_dict[postcode] = LSOA_codes_here
    
    hospital_name = df_nearest_hospitals_here['Nearest hospital name'].iloc[0]
    hosptial_name_dict[postcode] = hospital_name
    
    # Update lists for the dataframe:
    ivt_hospital_names_list.append(hospital_name)
    n_lsoas_list.append(len(LSOA_codes_here))

In [6]:
lsoas_per_ivt_hospital = pd.DataFrame(
    np.stack([ivt_hospital_names_list, n_lsoas_list], axis=-1),
    columns=['IVT hospital', 'Number of LSOAs']
)

In [8]:
lsoas_per_ivt_hospital.sort_values('IVT hospital').to_csv('lsoas_total_per_ivt_hospital.csv', index=False)

## Examine the big .geojson

What order are the LSOA shapes stored in? It seems to be sorted by LSOA11CD.

In [14]:
with open('./LSOA_(Dec_2011)_Boundaries_Super_Generalised_Clipped_(BSC)_EW_V3_reduced4_simplified.geojson') as f:
    geojson_ew = json.load(f)

In [15]:
geojson_ew.keys()

dict_keys(['type', 'name', 'crs', 'features'])

In [16]:
big_geojson_order = []

for i in range(len(geojson_ew['features'])):
    big_geojson_order.append(geojson_ew['features'][i]['properties']['LSOA11NM'])

In [17]:
geojson_ew['features'][0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-0.095, 51.521],
    [-0.097, 51.522],
    [-0.098, 51.521],
    [-0.1, 51.517],
    [-0.095, 51.515],
    [-0.095, 51.521]]]},
 'properties': {'LSOA11NM': 'City of London 001A'}}

## Match big .geojson info with regions

In [19]:
for group in code_dict.keys():
    lsoa_here = code_dict[group]
    features_here = []
    for lsoa in lsoa_here:
        # Find where it is in the big geojson:
        try:
            ind = big_geojson_order.index(lsoa)
            # inds_for_geojson.append(ind)
            features_here.append(geojson_ew['features'][ind])
        except ValueError:
            print(f'Problem with {group}, LSOA {lsoa}')
    
    features_here = FeatureCollection(features_here)
            
#     geojson_here = {}
#     geojson_here['type'] = geojson_ew['type']
#     geojson_here['crs'] = geojson_ew['crs']
#     geojson_here['features'] = features_here
    
    # Now save the file as something new:
    # group_name = group_dict[group]
    # group_name = group_name.replace(' ', '~')
    save_name = 'LSOA_' + group + '.geojson'
    
    with open('./nearest_hospital_geojson/'+save_name, 'w', encoding='utf-8') as f:
        json.dump(features_here, f, ensure_ascii=False)

### Import one of the newly-made .geojson

In [4]:
# with open('./lhb_scn_geojson/LSOA_South~West.geojson') as f:
with open('./nearest_hospital_geojson/LSOA_EX25DW.geojson') as f:
    geojson_devon = json.load(f)

In [5]:
# Create a map
devon_map = folium.Map(location=[51, -3.5],
                        zoom_start=8,
                        tiles='cartodbpositron')
# Add choropleth
folium.Choropleth(geo_data=geojson_devon,
                  highlight=True).add_to(devon_map)

# Generate map
devon_map